# Задание
Обучить и оценить модель по данным load_boston с использованием сетки параметров ```ParamGridBuilder``` и ```Pipeline```. Оценивается модель при помощи ```MAE``` 

Датасет взял [здесь](https://www.kaggle.com/datasets/puxama/bostoncsv?resource=download), так как в sklearn его больше нет

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=da899fb3ff6b02d8dd2457ac9959d5995f14bce368a909995299b6da44beba29
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
import requests
import pandas as pd

from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

Создаём Spark-сессию:

In [3]:
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_7")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()

Скачиваем данные с Google Drive. Так они выглядят в Pandas DataFrame:

In [4]:
train_response = requests.get('https://drive.google.com/uc?id=1YWYuawetKJlr9aJS4YoI1vQ2RIelQtOf')
with open('Boston.csv', 'wb') as file:
    file.write(train_response.content)
dataset = pd.read_csv('Boston.csv')
dataset.rename(columns={'medv': 'target'}, inplace=True)
dataset.head(5)

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,target
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


А так в Spark DataFrame:

In [5]:
spark_dataset = spark.createDataFrame(dataset)
spark_dataset.show(5)

+----------+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+------+
|Unnamed: 0|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|target|
+----------+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+------+
|         1|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|  24.0|
|         2|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|  21.6|
|         3|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|  34.7|
|         4|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|  33.4|
|         5|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|  36.2|
+----------+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+------+
only showing top 5 rows



Подготовим и обучим модель. В пайплайне используем:
- ```VectorAssembler``` - трансформер, который объединит признаки из ```feature_columns``` в один плотный вектор значений. Построенный вектор будет иметь имя 'features'
- ```StandardScaler``` - стандартизация
- ```RandomForestRegressor``` - алгоритм случайного леса

В переборе параметров с ```помощью param_grid``` рассморим следующие:
- ```numTrees``` - количество деревьев в лесу (50, 100 или 200 деревьев)
- ```maxDepth``` - максимальная глубина деревьев в лесу (5, 10 или 20 уровней)
- ```withMean``` - булевый флаг, указывающий следует ли центрировать данные перед обучением модели (True или False)

Так же для уверенности прогноза при помощи ```CrossValidator``` применим кросс-валидацию на 3 фолда

In [6]:
feature_columns = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 
                   'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat']

pipeline = Pipeline(stages=[
    VectorAssembler(inputCols=feature_columns, outputCol='features'),
    StandardScaler(inputCol='features', outputCol='scaled_features'),
    RandomForestRegressor(featuresCol='features', labelCol='target')
])

# Разделение на обучающую и тестовую выборки
train, test = spark_dataset.randomSplit([0.7, 0.3], seed=42)

# Конфигурация гиперпараметров
param_grid = ParamGridBuilder() \
    .addGrid(RandomForestRegressor.numTrees, [50, 100, 200]) \
    .addGrid(RandomForestRegressor.maxDepth, [5, 10, 20]) \
    .addGrid(StandardScaler.withMean, [True, False]) \
    .build()

# Кросс-валидация
cross_validator = CrossValidator(estimator=pipeline, 
                                 estimatorParamMaps=param_grid,
                                 evaluator=RegressionEvaluator(predictionCol='prediction', labelCol='target', metricName='mae'),
                                 numFolds=3,
                                 seed=42)

# Обучим модель на тренировочных данных
model = cross_validator.fit(train.orderBy(F.rand()))

In [7]:
# Получим прогнозов для тренировочных и тестовых данных
train_predictions = model.transform(train)
test_predictions = model.transform(test)

# Так мы оценим качество медели 
mae_evaluator_test = RegressionEvaluator(predictionCol='prediction', labelCol='target', metricName='mae')
mae_test = mae_evaluator_test.evaluate(test_predictions)

mae_evaluator_train = RegressionEvaluator(predictionCol='prediction', labelCol='target', metricName='mae')
mae_train = mae_evaluator_train.evaluate(train_predictions)

print(f'MAE на train-выборке: {mae_train:.2f}')
print(f'MAE на test-выборке:  {mae_test:.2f}')

MAE на train-выборке: 1.93
MAE на test-выборке:  2.41


результат на тестовой выборке получился ощутимо лучше, чем в условном задании без тюнинга, т.к. там результат был таким:

    Scores:: 
        train: 1.7964428791089762, 
        test: 2.6747956161632347

Вывод: наша модель работает и она молодец :)